## Notebook 6: Decision Tree

Este notebook está basado en varios videos sobre Machine Learning elaborados por <i>Google Developers</i> en Youtube. Ver: <i><a href="https://www.youtube.com/watch?v=cKxRvEZd3Mw"> Machine Learning Recipes</a></i>.

Este notebook se divide en 2 partes:
- En la primera parte se aplica un algoritmo de Árbol de Decisión para aprender un modelo capaz de clasificar los datos del dataset "Titanic" (visto en un notebook anterior). Compararemos su precision y recall con otros algoritmos de clasificación (kNN, Regresión Logística y Random Forest).
- En la segunda parte se implenta un algoritmo de Árbol de Decisión <i>from scratch</i> (desde cero) en Python para identificar sus características más precisamente.

# PARTE I

## 0. Preparar el dataset 'Titanic' 

Como en un notebook anterior, vamos a utilizar el dataset 'Titanic' con el cuál se tratar de predicir si una persona va a sobrevivir o no dadas algunas características (edad, genero, n° de niños, etc.).

En primera instancia, vamos a aplicar nuevamente todos los procesos para preparar el dataset y cargarlo en un DataFrame.

In [24]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np  
import matplotlib.pyplot as plt  
import pandas as pd 

def age_approx(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):
        if Pclass == 1:
            return 37
        elif Pclass == 2:
            return 29
        else:
            return 24
    else:
        return Age

# Read dataset to pandas dataframe
url = 'https://raw.githubusercontent.com/BigDataGal/Python-for-Data-Science/master/titanic-train.csv'
titanic = pd.read_csv(url)
titanic.columns = ['PassengerId','Survived','Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']
titanic_data = titanic.drop(['PassengerId','Name','Ticket','Cabin'], 1)

titanic_data['Age'] = titanic_data[['Age', 'Pclass']].apply(age_approx, axis=1)
titanic_data.isnull().sum()

titanic_data.dropna(inplace=True)
titanic_data.isnull().sum()

gender = pd.get_dummies(titanic_data['Sex'],drop_first=True)
embark_location = pd.get_dummies(titanic_data['Embarked'],drop_first=True)

titanic_data.drop(['Sex', 'Embarked'],axis=1,inplace=True)

titanic_dmy = pd.concat([titanic_data,gender,embark_location],axis=1)

X = titanic_dmy.ix[:,(1,2,3,4,5,6,7,8)].values
y = titanic_dmy.ix[:,0].values


## 1. Entrenar un clasificador de tipo DecisionTree 

In [25]:

from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state=25)

from sklearn import tree
#Entrenamiento
clf = tree.DecisionTreeClassifier().fit(X_train,y_train)
#Predicciones
y_pred = clf.predict(X_test)
#Evaluación del rendimiento del clasificador
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)
#Print de la matriz de confusión
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

[[132  32]
 [ 23  80]]
             precision    recall  f1-score   support

          0       0.85      0.80      0.83       164
          1       0.71      0.78      0.74       103

avg / total       0.80      0.79      0.80       267



Los modelos de clasificación de tipo DecisionTree tienen una caracteristica interesante (en comparación con otros algoritmos de clasificación): se puede <b>visualizar</b> el modelo y <b>entender</b> cómo toma decisiones. El código de la celda siguiente permite visualizar el modelo que hemos aprendidó.

In [26]:
from sklearn.externals.six import StringIO
import pydot #sudo python3.6 -m pip install pydot

dot_data = StringIO()

features=['Pclass','Age','SibSp','Parch','Fare','male','Q', 'S']
classes=['yes','no']
tree.export_graphviz(clf,out_file=dot_data,feature_names=features,class_names=classes, filled=True, 
                     rounded=True, impurity=False)

graph = pydot.graph_from_dot_data(dot_data.getvalue())
graph[0].write_png('modelo.png')

<img src="modelo.png">Modelo DecisionTree</img>

In [27]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(X_train, y_train)

In [28]:
#Predicciones
y_pred = clf.predict(X_test)
#Evaluación del rendimiento del clasificador
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)
#Print de la matriz de confusión
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

[[137  27]
 [ 31  72]]
             precision    recall  f1-score   support

          0       0.82      0.84      0.83       164
          1       0.73      0.70      0.71       103

avg / total       0.78      0.78      0.78       267



<b>Pregunta</b>:
- Para resolver el problema de clasificación del dataset 'Titanic', ¿Cuál algoritmo parece funcionar mejor entre kNN, Regresión Logística, DecisionTree o Random Forest?<br>
<b>R:</b>Random Forest
- ¿Por qué Random Forest funciona mejor que Decision Tree? ¿Cuál es la particularidad de Random Forest?<br>
<b>R:</b> es una combinación de árboles predictores tal que cada árbol depende de los valores de un vector aleatorio probado independientemente y con la misma distribución para cada uno de estos. Es una modificación sustancial de bagging que construye una larga colección de árboles no correlacionados y luego los promedia.

# PARTE 2

## Implementar un algoritmo DecisionTree desde cero

Este ejercicio se base en la video: https://www.youtube.com/watch?v=LDRbO9a6XPU

Para entender más precisamente cómo y qué <b>aprenden</b> los algoritmos de tipo Decision Tree, implantamos a continuación un algoritmo desde cero en Python. En camino, presentaremos los conceptos de <b>Gini impurity</b> e <b>Information Gain</b>.

<b>a. Algoritmos de tipo Decision Tree: El ejemplo de CART</b>

Existen varias variaciones de algoritmos Decision Tree, entre las cuales: ID3, C4.5, C5.0, CART <a href="https://en.wikipedia.org/wiki/Decision_tree">(ver wikipedia)</a>
Cada algoritmo comparta la misma idea: a cada nivel del árbol de decisión, el modelo formula una pregunta permitiendo poco a poco de llegar a una predicción. ¿Cómo el algoritmo sabe qué preguntas hacer y en qué orden? Es lo que vamos a responder a través del ejemplo de CART. 

CART (Classification and Regression Trees) inicializa la raíz del árbol con el dataset de entrenamiento completo (ver Figura). Luego busca dividir los datos del dataset con una pregunta. Los nodos siguientes reciben solamente los datos que corresponde a la respuesta. Se reitera este proceso hasta desenredar totalmente los datos (cada hoja del árbol debería tener idealmente un solo tipo de label).

<img src="./CART.png"></img>

En nuestro ejemplo, el nodo 1 está totalmente desenredado (label: "Grape"). El nodo 2 tiene dos labeles, entonces preguntamos otra pregunta:

<img src="./CART2.png"></img>

Para construir un árbol eficiente, el punto importante es identificar qué preguntas formular y cuando. Por lo tanto, necesitamos <b>cuantificar</b> en qué medida una pregunta permite desenredar los labeles. Para hacer eso se utiliza 2 métricas:
- el coeficiente de '<b>Gini impurity</b>': mide que tan desenredados están los labeles de un nodo.
- el coeficiente de '<b>Information Gain</b>': mide cuánto una pregunta permite bajar el 'Gini impurity'.

<img src="./CART3.png"></img>

Utilizaremos estas métricas para estimar qué preguntas hacer.

<b>b. ¿Qué preguntas formular?</b>

<img src="./CART4.png"></img>

Para saber qué preguntas formular, cada nodo itera sobre las características de los datos a su disposición y define una lista de preguntas posibles.

<b>c. Partición del dataset</b>

Una vez una pregunta elegida, se divide los datos en dos según la respuesta a la pregunta.

<img src="./CART5.png"></img>

<b>d. Coeficiente de Gini impurity</b>

El coeficiente de Gini impurity representa la probabilidad de ser incorrecto si asigna aleatoriamente una etiqueta a un ejemplo del mismo conjunto. Por ejemplo, en los dos ejemplos siguientes: ¿Cuál es la probabilidad de equivocarse si asignamos una etiqueta del recipiente B a un dato del recipiente A?

Ejemplo 1:
<img src="./CART-6.png"></img>

Ejemplo 2:
<img src="./CART-7.png"></img>

<b>e. Information Gain </b>

La métrica de Information Gain permite medir qué pregunta optimiza el coeficiente de Gini impurity.

Por cada nodo, empezamos por medir el coeficiente de Gini impurity de los labeles disponibles. Luego, por cada pregunta calculamos el coeficiente de Gini impurity de los dos sub-conjuntos de datos obtenidos.

<img src="CART-8.png"></img>

Calculamos la incerteza (<i>impurity</i>) promedia ponderada para los dos subconjuntos de datos obtenidos. Por ejemplo:

<img src="CART-9.png"></img>

Finalmente, conservamos la pregunta que permite optimizar la ganancia de información (Information Gain). En nuestro ejemplo:

<b>Information Gain = 0.64 - 0.15 = 0.14</b>

## Código del Decision Tree desde cero...

<b>a. Dataset</b>

Tenemos un dataset muy simple, constituido por 2 características y un label. En nuestro escenario, se trata de predecir cuál es la fruta a partir de su color y tamaño. Están libre de agregar nuevos ejemplos en el dataset para experimentar los cambios generados en el modelo...  

In [29]:
# Toy dataset.
# Format: each row is an example.
# The last column is the label.
# The first two columns are features.
# Feel free to play with it by adding more features & examples.
# Interesting note: I've written this so the 2nd and 5th examples
# have the same features, but different labels - so we can see how the
# tree handles this case.
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

Creamos una variable para indicar los nombres de cada columna del dataset:

In [30]:
# Column labels.
# These are used only to print the tree.
header = ["color", "diameter", "label"]

En el dataset, pueden observar que nuestros datos no son linealmente separable. Una observación de manzana y una observación de limón tienen las mismas características (segundo y quinto ejemplos del dataset). Vamos a observar cómo el algoritmo Decision Tree gestiona estos ejemplos.

<b>c. Funciones de utilidad</b>:

A continuación creamos una serie de funciones Python para facilitar la observación de nuestro dataset de entrenamiento. Por cada función, creamos una pequeña demo en la celda siguiente.

In [31]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])

In [32]:
#######
# Demo:
unique_vals(training_data, 0)
#unique_vals(training_data, 1)
#######

{'Green', 'Red', 'Yellow'}

In [33]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [34]:
#######
# Demo:
class_counts(training_data)
#######

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [35]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)
#######
# Demo:
is_numeric(7)
# is_numeric("Red")
#######

True

In [36]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [37]:
#######
# Demo:
# Let's write a question for a numeric attribute
Question(1, 3)

Is diameter >= 3?

In [38]:
# How about one for a categorical attribute
q = Question(0, 'Green')
q

Is color == Green?

In [39]:
# Let's pick an example from the training set...
example = training_data[0]
# ... and see if it matches the question
q.match(example) # this will be true, since the first example is Green.
#######

True

In [40]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [41]:
#######
# Demo:
# Let's partition the training data based on whether rows are Red.
true_rows, false_rows = partition(training_data, Question(0, 'Red'))
# This will contain all the 'Red' rows.
true_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [42]:
# This will contain everything else.
false_rows
#######

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [43]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [44]:
#######
# Demo:
# Let's look at some example to understand how Gini Impurity works.
#
# First, we'll look at a dataset with no mixing.
no_mixing = [['Apple'],
              ['Apple']]
# this will return 0
gini(no_mixing)

0.0

In [45]:
# Now, we'll look at dataset with a 50:50 apples:oranges ratio
some_mixing = [['Apple'],
               ['Orange']]
# this will return 0.5 - meaning, there's a 50% chance of misclassifying
# a random example we draw from the dataset.
gini(some_mixing)

0.5

In [46]:
# Now, we'll look at a dataset with many different labels
lots_of_mixing = [['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Grapefruit'],
                  ['Blueberry']]
# This will return 0.8
gini(lots_of_mixing)
#######

0.7999999999999998

In [47]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [48]:
#######
# Demo:
# Calculate the uncertainy of our training data.
current_uncertainty = gini(training_data)
current_uncertainty

0.6399999999999999

In [49]:
# How much information do we gain by partioning on 'Green'?
true_rows, false_rows = partition(training_data, Question(0, 'Green'))
info_gain(true_rows, false_rows, current_uncertainty)

0.1399999999999999

In [50]:
# What about if we partioned on 'Red' instead?
true_rows, false_rows = partition(training_data, Question(0,'Red'))
info_gain(true_rows, false_rows, current_uncertainty)

0.37333333333333324

In [51]:
# It looks like we learned more using 'Red' (0.37), than 'Green' (0.14).
# Why? Look at the different splits that result, and see which one
# looks more 'unmixed' to you.
true_rows, false_rows = partition(training_data, Question(0,'Red'))

# Here, the true_rows contain only 'Grapes'.
true_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [52]:
# And the false rows contain two types of fruit. Not too bad.
false_rows

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [53]:
# On the other hand, partitioning by Green doesn't help so much.
true_rows, false_rows = partition(training_data, Question(0,'Green'))

# We've isolated one apple in the true rows.
true_rows

[['Green', 3, 'Apple']]

In [54]:
# But, the false-rows are badly mixed up.
false_rows
#######

[['Yellow', 3, 'Apple'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Yellow', 3, 'Lemon']]

In [55]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [56]:
#######
# Demo:
# Find the best question to ask first for our toy dataset.
best_gain, best_question = find_best_split(training_data)
best_question
# FYI: is color == Red is just as good. See the note in the code above
# where I used '>='.
#######

Is diameter >= 3?

In [57]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [58]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [59]:
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)


In [60]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [61]:
my_tree = build_tree(training_data)

In [62]:
print_tree(my_tree)

Is diameter >= 3?
--> True:
  Is color == Yellow?
  --> True:
    Predict {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict {'Apple': 1}
--> False:
  Predict {'Grape': 2}


In [63]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)


In [64]:
#######
# Demo:
# The tree predicts the 1st row of our
# training data is an apple with confidence 1.
classify(training_data[0], my_tree)
#######

{'Apple': 1}

In [65]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [66]:
#######
# Demo:
# Printing that a bit nicer
print_leaf(classify(training_data[0], my_tree))
######

{'Apple': '100%'}

In [67]:
#######
# Demo:
# On the second example, the confidence is lower
print_leaf(classify(training_data[1], my_tree))
#######

{'Apple': '50%', 'Lemon': '50%'}

In [68]:
# Evaluate
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [69]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Apple. Predicted: {'Apple': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Apple': '50%', 'Lemon': '50%'}


## Trabajo práctico

1) Construir un pequeño dataset de entrenamiento para resolver un problema de clasificación simple (usted lo elige)

2) Utilizar la implementación de CART (código anterior) para medir los coeficientes de 'Gini impurity' e 'Information Gain' para ciertas preguntas y ciertos nodos

3) Pedir a un compañero de construir un pequeño dataset de test siguiendo el mismo formato que el dataset de entrenamiento

4) Visualizar el arbol de decisión aprendido

5) Evaluar la Precision y el Recall de su modelo


In [18]:
import pandas as pd  
import numpy as np 

import pandas as pd  
import numpy as np 
from sklearn.model_selection import train_test_split

dataset = pd.read_csv('bill_authentication.csv')
X = dataset.iloc[:, 0:4].values  
y = dataset.iloc[:, 4].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [19]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])


In [20]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        if row not in counts:
            counts[row] = 0
        counts[row] += 1
    return counts

In [21]:
print(unique_vals(X_train, 0))
print(class_counts(y_train))



{0.24261, 1.2279, 1.6349, 3.9433, 4.3848, 2.3164, 2.9695, 2.6463, 4.052, 5.1213, 6.8248, 3.5152, 3.8384, 4.7072, 5.0214, 2.9939999999999998, 3.7439999999999998, 0.74841, -1.1188, -1.5075, 4.6464, 0.007124899999999999, -2.1674, -2.9662, 4.9923, 5.2012, 5.2620000000000005, 1.7425, -1.8215, -0.3489, -6.7526, -5.2406, -5.1216, 0.7225199999999999, 2.7213, -1.8076, 2.8523, -1.2528, 3.9362, -3.9204, 3.6922, -0.60975, 1.2198, 1.8799, 0.8829799999999999, 0.11685999999999999, 1.5691, 1.8373, 1.7748, 2.2123, 2.4486, 0.049175, -3.5681, -3.5741, -3.0193, 4.988, -2.9019999999999997, 4.6361, -2.7419, -0.60254, -1.0116, -1.4217, -6.4247, 1.2247, 1.6408, 0.9270299999999999, -0.64326, 2.6682, 3.1896, -2.0891, -2.0754, 1.9321, -2.8391, -2.5754, 1.7331, -0.40951, -0.7784800000000001, 0.26877, 0.19080999999999998, 0.062525, 0.3434, 2.5068, 2.1943, -1.0555, 3.583, -3.9594, 3.2294, 3.6181, 3.4246, 3.577, -2.8267, -1.7582, -1.8969, -1.682, -1.4106, 6.5633, 2.0466, 2.9233, 3.2051, -4.8554, 3.2692, 3.0329, 1.01

In [70]:
my_tree = build_tree(X_train)

In [96]:
print_tree(my_tree)

IndexError: list index out of range

In [85]:
Expected=[]
Predicted=[]
for row in X_test:
    Expected.append(row[-1])
    
    Predicted.append(print_leaf(classify(row, my_tree)))
    
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: 1.0. Predicted: {0.0: '100%'}
Actual: 0.0. Predicted: {1.0: '100%'}
Actual: 1.0. Predicted: {1.0: '100%'}
Actual: 1.0. Predicted: {1.0: '100%'}
Actual: 1.0. Predicted: {1.0: '100%'}
Actual: 1.0. Predicted: {0.0: '100%'}
Actual: 1.0. Predicted: {1.0: '100%'}
Actual: 0.0. Predicted: {0.0: '100%'}
Actual: 1.0. Predicted: {1.0: '100%'}
Actual: 1.0. Predicted: {1.0: '100%'}
Actual: 1.0. Predicted: {1.0: '100%'}
Actual: 1.0. Predicted: {1.0: '100%'}
Actual: 1.0. Predicted: {1.0: '100%'}
Actual: 1.0. Predicted: {1.0: '100%'}
Actual: 1.0. Predicted: {1.0: '100%'}
Actual: 1.0. Predicted: {1.0: '100%'}
Actual: 1.0. Predicted: {1.0: '100%'}
Actual: 1.0. Predicted: {1.0: '100%'}
Actual: 1.0. Predicted: {0.0: '100%'}
Actual: 1.0. Predicted: {1.0: '100%'}
Actual: 1.0. Predicted: {1.0: '100%'}
Actual: 1.0. Predicted: {1.0: '100%'}
Actual: 0.0. Predicted: {0.0: '100%'}
Actual: 1.0. Predicted: {1.0: '100%'}
Actual: 0.0. Predicted: {0.0: '100%'}
Actual: 0.0. Predicted: {0.0: '100%'}
Actual: 0.0.

In [95]:
PredictedFixed=[]
for x in Predicted:
    index=str(x)
    index=index[1:4]
    index=float(index)
    PredictedFixed.append(index)
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Expected, PredictedFixed)
print(confusion_matrix)
#Print de la matriz de confusión
from sklearn.metrics import classification_report
print(classification_report(Expected, PredictedFixed))

[[ 49  17]
 [ 20 181]]
             precision    recall  f1-score   support

        0.0       0.71      0.74      0.73        66
        1.0       0.91      0.90      0.91       201

avg / total       0.86      0.86      0.86       267



In [75]:
feature_cols=list(dataset)
feature_cols.remove('Class')

In [78]:

from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state=25)

from sklearn import tree
#Entrenamiento
clf = tree.DecisionTreeClassifier().fit(X_train,y_train)
#Predicciones
y_pred = clf.predict(X_test)
#Evaluación del rendimiento del clasificador
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)
#Print de la matriz de confusión
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

[[133  31]
 [ 22  81]]
             precision    recall  f1-score   support

          0       0.86      0.81      0.83       164
          1       0.72      0.79      0.75       103

avg / total       0.81      0.80      0.80       267



In [79]:
from sklearn.externals.six import StringIO
import pydot #sudo python3.6 -m pip install pydot

dot_data = StringIO()

classes=['1','0']
tree.export_graphviz(clf,out_file=dot_data,feature_names=feature_cols,class_names=classes, filled=True, 
                     rounded=True, impurity=False)

graph = pydot.graph_from_dot_data(dot_data.getvalue())
graph[0].write_png('arbolAprendido.png')

ValueError: Length of feature_names, 4 does not match number of features, 8